In [1]:
######################
#Librerías necesarias
######################

#Librerías de python y google colab
import os
import numpy as np
import matplotlib.pyplot as plt
import tarfile
import random
import copy
import itertools
from google.colab import drive

#Pandas
import pandas as pd

#Pytorch
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from skimage import io
from torch.utils.data.dataset import Subset
from torchvision.transforms.transforms import FiveCrop
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import sampler, random_split
from torch.utils.data.dataloader import DataLoader
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.activation import ReLU
from torchsummary import summary

#Scikit Learn
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [2]:
#Asociar la cuenta de Google Drive con el cuaderno para leer los archivos .tar

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Extracción de los archivos .tar

data_dir = "/content/data"

#Extracción de train (cambiar directorio)
tar = tarfile.open("/content/drive/MyDrive/TFG/train_set.tar")
tar.extractall(data_dir)
tar.close()

#Extraemos test (cambiar directorio)
tar = tarfile.open("/content/drive/MyDrive/TFG/val_set.tar")
tar.extractall(data_dir)
tar.close()

#Cálculo de la media y desviación estándar

In [5]:
#Función que calcula la media y la desviación estándar 

def media_std(loader):

  #VAR[x] = E[X**2] - E[x]**2
  suma_canales, sumacuadradocanales, num_batches = 0,0,0

  for i, (data, target) in enumerate(loader):
    
    suma_canales += torch.mean(data, dim=[0,2,3])#Media para el ancho y alto
    sumacuadradocanales += torch.mean(data**2, dim =[0,2,3]) #E[x**2]
    
    num_batches +=1
  
  #Dividimos la suma entre el número de batches para obtener la media
  media = suma_canales/num_batches
  #Hacemos la raíz cuadrada de la varianza para obtener la desviación estándar
  std = (sumacuadradocanales/num_batches - media**2)**0.5

  return media, std


In [6]:
#Clase AffectNet para el conjunto de test

class AffectNetDataset_test(Dataset):
  def __init__(self, csv_file, root_dir, transform = None):

    self.annotations= pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    
    img_path=os.path.join(self.root_dir, self.annotations.iloc[index,0])
    image = io.imread(img_path)
    y_label = torch.tensor(int(self.annotations.iloc[index,1]))
    
    img = self.transform(image)
    
    return (img, y_label)

In [8]:



train_dir= "/content/data/train_set/images"

#Creamos un "dumb" dataset que se usa para obtener la media  y la std
dumb_dataset = AffectNetDataset_test(csv_file='AffectNet.csv', root_dir = train_dir, transform = transforms.ToTensor())
dumb_loader = DataLoader(dataset = dumb_dataset, batch_size = 256, shuffle = True)

mean,std = media_std(dumb_loader)
print(mean)
print(std)

tensor([0.5694, 0.4460, 0.3912])
tensor([0.2747, 0.2446, 0.2383])


# Cálculo de los pesos

In [9]:

#Para el cáclulo de los pesos, es necesario disponer de las etiquetas sin crear una base de datos

train_annotations_path = "/content/data/train_set/annotations"
train_dir = os.listdir(train_annotations_path)
val_annotations_path = "/content/data/val_set/annotations"
dir_val = os.listdir(val_annotations_path)


#Si el archivo contiene exp.npy, metemos su ruta en el array 
train_npy=[]
val_npy=[]

for i in range(len(train_dir)):
  if 'exp.npy' in train_dir[i]:
    train_npy.append(train_dir[i])

for i in range(len(dir_val)):
  if 'exp.npy' in dir_val[i]:
    val_npy.append(dir_val[i])

#Ordenamos el array mirando su primer elemento
train_npy.sort(key=lambda x: [int(x[0:-8])])
print(len(train_npy))

val_npy.sort(key=lambda x: [int(x[0:-8])])
print(len(val_npy))


#Función que procesa el archivo npy y obtiene su valor
def npy_loader(path):
   sample = np.load(path)
   return sample

#Guardamos las etiquetas en el array clases
train_classes=[]
for item in range(len(train_npy)):
  train_classes.append(npy_loader(train_annotations_path + "/" + train_npy[item]))

val_classes=[]
for item in range(len(val_npy)):
  #print(item)
  val_classes.append(npy_loader(val_annotations_path + "/" + val_npy[item]))


287651
3999


In [10]:
#Se procede de la misma forma con las imágenes

train_images_path = "/content/data/train_set/images"
train_images = os.listdir(train_images_path)
train_images.sort(key=lambda x: [int(x[0:-4])])

val_images_path = "/content/data/val_set/images"
val_images = os.listdir(val_images_path)
val_images.sort(key=lambda x: [int(x[0:-4])])

In [18]:
#Creación del archivo CSV que contiene la ruta a las imágenes y su etiqueta

import pandas as pd

Num_total_train=len(train_images)


#Guardamos el nombre del archivo y su etiqueta en un dataframe de pandas
data={}
data['images']= train_images
data['label']= train_classes

print(len(data['images']))
print(len(data['label']))
data_df = pd.DataFrame(data, columns=['images', 'label'])
print(data_df)

data_df.to_csv("AffectNet.csv", index=False, header = False)


val_data={}
val_data['images']= val_images
val_data['label']= val_classes

val_df = pd.DataFrame(val_data, columns=['images', 'label'])
print(val_df)

val_df.to_csv("AffectNet_val.csv", index=False, header = False)








287651
287651
            images label
0            0.jpg     1
1            1.jpg     0
2            2.jpg     0
3            3.jpg     1
4            5.jpg     6
...            ...   ...
287646  414792.jpg     2
287647  414793.jpg     0
287648  414794.jpg     2
287649  414795.jpg     2
287650  414796.jpg     1

[287651 rows x 2 columns]
        images label
0        0.jpg     0
1        1.jpg     0
2        2.jpg     4
3        3.jpg     0
4        4.jpg     2
...        ...   ...
3994  5489.jpg     0
3995  5490.jpg     6
3996  5492.jpg     6
3997  5494.jpg     3
3998  5495.jpg     2

[3999 rows x 2 columns]


In [21]:
#Array que contiene las etiquetas 

train_labels= []

#Guardamso las etiquetas en un array
for idx, (data, label) in enumerate(dumb_dataset):
  train_labels.append(label.item())

#Creamos una serie que cuenta las apariciones de cada etiqueta 
train_counts = pd.Series(train_labels).value_counts().sort_index()
print(train_counts)

#Calculamos los pesos como la inversa
train_weights= (1/torch.Tensor(train_counts))
print(train_weights)





  

0     74874
1    134414
2     25459
3     14090
4      6378
5      3803
6     24882
7      3750
dtype: int64
tensor([1.3356e-05, 7.4397e-06, 3.9279e-05, 7.0972e-05, 1.5679e-04, 2.6295e-04,
        4.0190e-05, 2.6667e-04])
